In [1]:
import pandas as pd
import surgeo

In [109]:
import geopandas as gpd
import json 

In [76]:
finance_df = pd.read_csv('../../council_campaign_finance/data/located_data_subset_for_member_chapters.csv')

In [77]:
finance_df['first_name'] = finance_df['NAME'].apply(lambda x: x.split(' ')[1])
finance_df['last_name'] = finance_df['NAME'].apply(lambda x: x.split(' ')[0]).str.replace(',', '')

In [78]:
finance_df.sample(5)

,REFNO,DATE,matched_name,ZIP,latitude,longitude,OCCUPATION,EMPNAME,AMNT,NAME,first_name,last_name
4296,R0004375,1/11/2021,Henry L Butler,11221,40.6907,-73.9274,Retired,NaN,100.0,"Kalokoh, Jacqueline",Jacqueline,Kalokoh
7728,R0000041,11/12/2020,Robert Waterman,11435,40.7029,-73.8111,Dean,NYC Dept of Education,100.0,"Seabrooks, Gerald",Gerald,Seabrooks
1619,R0000420,11/15/2020,Yu Lin,11220,40.6412,-74.0133,Retired,NaN,100.0,"Chen, Yong Guang",Yong,Chen
4762,R0000521,10/19/2020,Robert Waterman,11552,40.6929,-73.6539,Self Employed,Borders Laurent Physician Llc,200.0,"Laurent, Bordes",Bordes,Laurent
6402,R0002315,1/11/2021,Michael D Hollingsworth,11213,40.6700,-73.9367,Unemployed,NaN,25.0,"Norvig, Isabella",Isabella,Norvig


In [79]:
fsg = surgeo.BIFSGModel()

In [80]:
sg = surgeo.SurgeoModel()

In [95]:
f = surgeo.FirstNameModel()

In [ ]:
def find_ethnicity(row):
    result = fsg.get_probabilities(
        pd.Series([row['first_name']]),
        pd.Series([row['last_name']]),
        pd.Series([row['ZIP']])
    )
    numeric_result = result.select_dtypes(include='number')
    predicted_ethnicity = numeric_result.idxmax(axis=1).iloc[0]
    return predicted_ethnicity

In [68]:
def find_ethnicity_sg(row):
    result = sg.get_probabilities(
        pd.Series([row['last_name']]),
        pd.Series([row['ZIP']])
    )
    numeric_result = result.select_dtypes(include='number')
    predicted_ethnicity = numeric_result.idxmax(axis=1).iloc[0]
    return predicted_ethnicity

In [81]:
finance_df['last_name'] = finance_df['last_name'].str.split('-').str[0]

In [82]:
finance_df['ethnicity'] = finance_df.apply(find_ethnicity, axis=1)

In [57]:
fsg.get_probabilities(pd.Series(['shouri']), pd.Series(['zheng']), pd.Series(['11228']))

,zcta5,first_name,surname,white,black,api,native,multiple,hispanic
0,11228,SHOURI,ZHENG,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
finance_df.isnull().sum()

REFNO              0
DATE               1
matched_name       0
ZIP                0
latitude           0
longitude          0
OCCUPATION       501
EMPNAME         2206
AMNT               0
NAME               0
first_name         0
last_name          0
ethnicity       2786
dtype: int64

In [85]:
fr_missing = finance_df[finance_df['ethnicity'].isnull()]

In [86]:
fr_missing['last_name'] = fr_missing['last_name'].str.split('-').str[0]

C:\Users\samtg\AppData\Local\Temp\ipykernel_7296\4134717725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fr_missing['last_name'] = fr_missing['last_name'].str.split('-').str[0]


In [90]:
fr_missing['ethnicity'] = fr_missing.apply(find_ethnicity_sg, axis=1)

C:\Users\samtg\AppData\Local\Temp\ipykernel_7296\3412879493.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fr_missing['ethnicity'] = fr_missing.apply(find_ethnicity_sg, axis=1)


In [93]:
fr_missing.isnull().sum()

REFNO              0
DATE               0
matched_name       0
ZIP                0
latitude           0
longitude          0
OCCUPATION       147
EMPNAME          699
AMNT               0
NAME               0
first_name         0
last_name          0
ethnicity       1363
dtype: int64

In [92]:
fr_missing.sample(5)

,REFNO,DATE,matched_name,ZIP,latitude,longitude,OCCUPATION,EMPNAME,AMNT,NAME,first_name,last_name,ethnicity
4254,R0000912,11/29/2020,Michael D Hollingsworth,10026,40.8019,-73.9531,Development,HFC,21.0,"Joseph, Jad",Jad,Joseph,black
3547,R0003130,1/11/2021,Michael D Hollingsworth,11249,40.6451,-73.9450,Attorney,Paul Weiss,10.0,"Hanna, Chantalle",Chantalle,Hanna,NaN
4147,R0002641,7/11/2020,Crystal Hudson,10025,40.7975,-73.9683,Unemployed,NaN,35.0,"Jinishian, Sam",Sam,Jinishian,NaN
3975,R0005521,1/11/2021,Crystal Hudson,11216,40.6794,-73.9496,Consultant,Self-Employed,35.0,"Ike, Ifeoma",Ifeoma,Ike,black
8069,R0000156,7/17/2020,Chi Osse,10034,40.8662,-73.9221,Unemployed,NaN,25.0,"Smith, Lucero",Lucero,Smith,white


In [94]:
still_missing = fr_missing[fr_missing['ethnicity'].isnull()]

In [96]:
def find_ethnicity_f(row):
    result = f.get_probabilities(
        pd.Series([row['first_name']]),
    )
    numeric_result = result.select_dtypes(include='number')
    predicted_ethnicity = numeric_result.idxmax(axis=1).iloc[0]
    return predicted_ethnicity

In [97]:
still_missing['ethnicity'] = fr_missing.apply(find_ethnicity_f, axis=1)

C:\Users\samtg\AppData\Local\Temp\ipykernel_7296\1392882880.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  still_missing['ethnicity'] = fr_missing.apply(find_ethnicity_f, axis=1)


In [98]:
still_missing.isnull().sum()

REFNO             0
DATE              0
matched_name      0
ZIP               0
latitude          0
longitude         0
OCCUPATION       43
EMPNAME         294
AMNT              0
NAME              0
first_name        0
last_name         0
ethnicity       292
dtype: int64

In [99]:
finance_df.dropna(subset=['ethnicity'], inplace=True)
fr_missing.dropna(subset=['ethnicity'], inplace=True)
still_missing.dropna(subset=['ethnicity'], inplace=True)

C:\Users\samtg\AppData\Local\Temp\ipykernel_7296\848753722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fr_missing.dropna(subset=['ethnicity'], inplace=True)
C:\Users\samtg\AppData\Local\Temp\ipykernel_7296\848753722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  still_missing.dropna(subset=['ethnicity'], inplace=True)


In [100]:
finance_df = pd.concat([finance_df, fr_missing, still_missing])

In [114]:
nyc_zip = gpd.read_file('../data/Modified Zip Code Tabulation Areas (MODZCTA).geojson')

In [119]:
zip_list = nyc_zip['zcta'].astype(str).str.split(', ').explode().tolist()

In [121]:
finance_df['in_nyc'] = finance_df['ZIP'].astype(str).isin(zip_list).astype(int)

In [125]:
finance_df.groupby('matched_name')['in_nyc'].mean()

matched_name
Alexa Aviles               0.802225
Chi Osse                   0.781440
Crystal Hudson             0.827247
Henry L Butler             0.955947
Jacqueline Painter         0.868644
Michael D Hollingsworth    0.843060
Reginald Swiney            0.920000
Robert Waterman            0.874659
Rodrigo Camarena           0.810093
Tahirah A Moore            0.893443
Yu Lin                     0.973301
Name: in_nyc, dtype: float64

In [127]:
finance_df.groupby('matched_name')['AMNT'].mean()

matched_name
Alexa Aviles                45.204487
Chi Osse                    48.663487
Crystal Hudson              82.366306
Henry L Butler              95.345815
Jacqueline Painter          43.487436
Michael D Hollingsworth     38.713173
Reginald Swiney            108.400000
Robert Waterman            105.212534
Rodrigo Camarena            47.120850
Tahirah A Moore             86.891393
Yu Lin                     101.344684
Name: AMNT, dtype: float64

In [133]:
finance_df

,REFNO,DATE,matched_name,ZIP,latitude,longitude,OCCUPATION,EMPNAME,AMNT,NAME,first_name,last_name,ethnicity,in_nyc
2,R0005771,1/20/2021,Michael D Hollingsworth,11238,40.6790,-73.9644,Editor,Conde Nast,25.0,"Abarbanel, Aliza",Aliza,Abarbanel,white,1
3,R0001644,6/11/2021,Robert Waterman,11221,40.6907,-73.9274,Publicist,C.C. Sullivan,25.0,"Abarbanel-Grossman, Alex",Alex,Abarbanel,black,1
4,R0003396,2/13/2021,Rodrigo Camarena,11232,40.6521,-74.0018,Assistant Location Manager,Universal Television,10.0,"Abbondanza, Jay",Jay,Abbondanza,white,1
5,R0001471,1/11/2021,Tahirah A Moore,11212,40.6625,-73.9145,NaN,Retired,20.0,"Abdul-Ali, Hassan",Hassan,Abdul,black,1
7,R0000481,5/21/2020,Alexa Aviles,94709,37.8784,-122.2655,Student,NaN,50.0,"Abelow, Sarah",Sarah,Abelow,white,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9479,R0004986,12/14/2020,Crystal Hudson,11218,40.6424,-73.9758,School mental health Consultan,Department of Health and Menta,36.0,"Zimiles, Eleni",Eleni,Zimiles,white,1
9480,R0006315,4/19/2021,Crystal Hudson,11218,40.6424,-73.9758,School mental health Consultan,Department of Health and Menta,100.0,"Zimiles, Eleni",Eleni,Zimiles,white,1
9485,R0007920,6/6/2021,Michael D Hollingsworth,11238,40.6790,-73.9644,Administrator,Clara Barton HIgh School,20.0,"Zisler, Jennifer",Jennifer,Zisler,white,1
9486,R0006515,5/12/2021,Crystal Hudson,11215,40.6669,-73.9828,Social Impact,MTV Entertainment Group,50.0,"Zorick, Maxwell",Maxwell,Zorick,white,1


In [148]:
finance_df[['NAME','OCCUPATION','EMPNAME','matched_name','AMNT','in_nyc']].sample(25)

,NAME,OCCUPATION,EMPNAME,matched_name,AMNT,in_nyc
1779,"Cohen, Adam",Marketing Manager,Springer Nature,Michael D Hollingsworth,1.0,0
9119,"Williams, Barbara",Coordinator,Nyc,Henry L Butler,25.0,1
4989,"Lin, Da",Unemployed,NaN,Yu Lin,10.0,1
3818,"Holbrook, Joshua",Staff Data Engineer,DoubleVerify Inc,Michael D Hollingsworth,100.0,1
3198,"Goheen, Jade",Not Employed,Not Employed,Michael D Hollingsworth,12.0,1
6943,"Porter, Kristina",Art Director,Cast and Crew Payroll Company,Michael D Hollingsworth,30.0,1
4470,"Khoury, Paige",Executive Assistant,Spencer Stuart,Crystal Hudson,20.0,0
2908,"Franklin, Remi",Director IT Audit,CohnReznick LLP,Henry L Butler,100.0,1
5783,"Mejia, Hector",NaN,Retired,Jacqueline Painter,10.0,1
2577,"Epstein, Andrew",Campaigns / Communications,Self-Employed,Michael D Hollingsworth,6.0,1


In [123]:
finance_with_ethnicity = finance_df.copy()

In [126]:
finance_with_ethnicity.to_csv('../data/finance_with_ethnicity.csv',index=False)